In [2]:
import os
# Set the correct path to https://github.com/PruneTruong/DenseMatching repository on your filesystem
module_path = "/home/klaus/eclipse_draft/DenseMatching/DenseMatching/"
# =======================================================================================++ #

In [3]:
import argparse
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import torch
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import sys
import matplotlib.cm as cm
import os.path as os

if module_path not in sys.path:
    sys.path.append(module_path)
from model_selection import model_type, pre_trained_model_types, select_model
from datasets.util import pad_to_same_shape
torch.set_grad_enabled(False)
from utils_flow.pixel_wise_mapping import remap_using_flow_fields
from utils_flow.visualization_utils import overlay_semantic_mask, make_sparse_matching_plot
from utils_flow.util_optical_flow import flow_to_image  
from models.inference_utils import estimate_mask
from utils_flow.flow_and_mapping_operations import convert_flow_to_mapping
from validation.utils import matches_from_flow
from admin.stats import DotDict 
%matplotlib inline

/home/klaus/anaconda3/envs/dense_matching_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Did not load moviepy


# Define the images to match

In [9]:
import os.path as osp

# choose model 
model = 'PDCNet'
pre_trained_model = 'megadepth'
flipping_condition = False 
global_optim_iter = 3
local_optim_iter = 7 
path_to_pre_trained_models = osp.join(module_path,'assets/pre_trained_models/')
    
if model not in model_type:
    raise ValueError('The model that you chose is not valid: {}'.format(model))
if pre_trained_model not in pre_trained_model_types:
    raise ValueError('The pre-trained model type that you chose is not valid: {}'.format(pre_trained_model))


# inference parameters for PDC-Net
network_type = model  # will only use these arguments if the network_type is 'PDCNet' or 'PDCNet_plus'
choices_for_multi_stage_types = ['d', 'h', 'ms']
multi_stage_type = 'h'
if multi_stage_type not in choices_for_multi_stage_types:
    raise ValueError('The inference mode that you chose is not valid: {}'.format(multi_stage_type))

confidence_map_R =1.0
ransac_thresh = 1.0
mask_type = 'proba_interval_1_above_10'  # for internal homo estimation
homography_visibility_mask = True
scaling_factors = [0.5, 0.6, 0.88, 1, 1.33, 1.66, 2]
compute_cyclic_consistency_error = True  # here to compare multiple uncertainty 

# usually from argparse
args = DotDict({'network_type': network_type, 'multi_stage_type': multi_stage_type, 'confidence_map_R': confidence_map_R, 
                'ransac_thresh': ransac_thresh, 'mask_type': mask_type, 
                'homography_visibility_mask': homography_visibility_mask, 'scaling_factors': scaling_factors, 
                'compute_cyclic_consistency_error': compute_cyclic_consistency_error})


In [10]:
# define network and load network weights
network, estimate_uncertainty = select_model(
    model, pre_trained_model, args, global_optim_iter, local_optim_iter,
    path_to_pre_trained_models=path_to_pre_trained_models)
estimate_uncertainty = True  
# here, we overwrite it, to also estimate uncertainty according to forward-backward for networks that do not predict a confidence measure

Model: PDCNet
Pre-trained-model: megadepth
GOCor: Local iter 7
GOCor: Global iter 3
/home/klaus/eclipse_draft/DenseMatching/DenseMatching/assets/pre_trained_models/PDCNet_megadepth.pth.tar


In [12]:
import os
import os.path as osp
import pandas as pd
import cv2 as cv
from tqdm import tqdm, trange
DS_FOLDER = '/home/klaus/eclipse_draft/KDD2022/kddbr-2022/'
PLOT = False # Used for debugging.
df = pd.read_csv(osp.join(DS_FOLDER,'public.csv'))
df['path'] = df['Filename']
df.loc[~pd.isna(df['North']), 'path'] = df.loc[~pd.isna(df['North']), 'path'].apply(lambda x: osp.join(DS_FOLDER, 'train','train', x))
df.loc[pd.isna(df['North']), 'path'] = df.loc[pd.isna(df['North']), 'path'].apply(lambda x: osp.join(DS_FOLDER, 'test','test', x))


def get_images(which_id):
    img = imageio.imread(df['path'].values[which_id], pilmode='RGB')
    query_image, reference_image = img[:,:120,:], img[:,120:,:] 
    query_image_shape = query_image.shape
    ref_image_shape = reference_image.shape
    if PLOT:
        fig, axis = plt.subplots(2, figsize=(10,5))
        axis[0].imshow(query_image)
        axis[0].set_title('Query image')
        axis[1].imshow(reference_image)
        axis[1].set_title('Reference image')
        plt.show()
    return query_image, reference_image


L = []

for which_id in trange(df.shape[0]):
    
    # INVERT!
    reference_image, query_image = get_images(which_id)
    
    fname = df['Filename'].values[which_id]
    alt = df['Altitude'].values[which_id]
    delta = df['Delta'].values[which_id]
    
    east = df['East'].values[which_id]
    north = df['North'].values[which_id]
    
    
    ref_image_shape = reference_image.shape
    query_image_shape = query_image.shape
    # convert the images to correct format to be processed by the network: torch Tensors, format B, C, H, W. 
    # pad both images to the same size, to be processed by network
    query_image_, reference_image_ = pad_to_same_shape(query_image, reference_image)

    # convert numpy to torch tensor and put it in right format
    query_image_ = torch.from_numpy(query_image_).permute(2, 0, 1).unsqueeze(0)
    reference_image_ = torch.from_numpy(reference_image_).permute(2, 0, 1).unsqueeze(0)

    if estimate_uncertainty:
        estimated_flow, uncertainty_components = network.estimate_flow_and_confidence_map(query_image_, reference_image_)
    else:
        if args.flipping_condition and 'GLUNet' in args.model:
            estimated_flow = network.estimate_flow_with_flipping_condition(query_image_, reference_image_,
                                                                           mode='channel_first')
        else:
            estimated_flow = network.estimate_flow(query_image_, reference_image_, mode='channel_first')
    # removes the padding
    estimated_flow = estimated_flow[:, :, :ref_image_shape[0], :ref_image_shape[1]]

    # convert to numpy and reformat
    estimated_flow_numpy = estimated_flow.squeeze().permute(1, 2, 0).cpu().numpy()

    # warp the query image according to the estimated flow
    warped_query_image = remap_using_flow_fields(query_image, estimated_flow_numpy[:, :, 0],
                                                 estimated_flow_numpy[:, :, 1]).astype(np.uint8)

    alpha = 0.5
    img_warped_overlay_on_target_masked = warped_query_image * alpha + reference_image * alpha
    if PLOT:
        plt.imshow(img_warped_overlay_on_target_masked.astype(np.uint8))
        plt.title('Warped query overlaid reference image')
        plt.show()
    # confidence estimation + visualization
    if not estimate_uncertainty: 
        raise ValueError
    if 'p_r' in uncertainty_components:
        uncertainty_key = 'p_r'  # 'inv_cyclic_consistency_error' 
    else:
        uncertainty_key = 'inv_cyclic_consistency_error'
    #'p_r', 'inv_cyclic_consistency_error' can also be used as a confidence measure
    # 'cyclic_consistency_error' can also be used, but that's an uncertainty measure
    min_confidence = 0.30
    confidence_map = uncertainty_components[uncertainty_key]
    confidence_map = confidence_map[:, :, :ref_image_shape[0], :ref_image_shape[1]]
    confidence_map_numpy = confidence_map.squeeze().detach().cpu().numpy()


    color = [255, 102, 51]
    confidence_map_numpy = confidence_map.squeeze().detach().cpu().numpy()
    confident_mask = (confidence_map_numpy > min_confidence).astype(np.uint8)
    confident_warped = overlay_semantic_mask(warped_query_image, ann=255 - confident_mask*255, color=color)

    # get the mask according to uncertainty estimation
    mask_type = 'proba_interval_1_above_10' # 'cyclic_consistency_error_below_2' 

    mask_padded = estimate_mask(mask_type, uncertainty_components) 
    if 'warping_mask' in list(uncertainty_components.keys()):
        # get mask from internal multi stage alignment, if it took place
        mask_padded = uncertainty_components['warping_mask'] * mask_padded

    # remove the padding
    mask = mask_padded[:, :ref_image_shape[0], :ref_image_shape[1]]

    # remove point that lead to outside the query image
    mapping_estimated = convert_flow_to_mapping(estimated_flow)
    mask = mask & mapping_estimated[:, 0].ge(0) & mapping_estimated[:, 1].ge(0) & \
    mapping_estimated[:, 0].le(query_image_shape[1] - 1) & mapping_estimated[:, 1].le(query_image_shape[0] - 1)

    mkpts_query, mkpts_ref = matches_from_flow(estimated_flow, mask)

    confidence_values = confidence_map.squeeze()[mask.squeeze()].cpu().numpy()
    sort_index = np.argsort(np.array(confidence_values)).tolist()[::-1]  # from highest to smallest
    confidence_values = np.array(confidence_values)[sort_index]
    mkpts_query = np.array(mkpts_query)[sort_index]
    mkpts_ref = np.array(mkpts_ref)[sort_index]

    if len(mkpts_query) > 5:
        M, mask = cv.findHomography(mkpts_query, mkpts_ref)
    else:
        M = np.zeros((3,3))
    confidence_map_numpy = confidence_map_numpy.reshape(-1)
    res = np.zeros(36)
    res[:9] = M.reshape(-1)
    res[9:20] =  np.quantile(estimated_flow_numpy[:,:,0],[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1])
    res[20:31] =  np.quantile(estimated_flow_numpy[:,:,1],[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1])
    res[31:36] = np.quantile(confidence_map_numpy,[0,0.1,0.25,0.5,1])
    
    L.append(res[None,:])
""" Time to concatenate the list of rows into a 2D array. """
X = np.concatenate(L,axis=0)
# Add appropriate column names
col = [f'mat_{i}' for i in range(9)] + [f'ef0_{i}' for i in range(11)]  + [f'ef1_{i}' for i in range(11)] + [f'conf_{i}' for i in range(5)]
# Create feature dataframe using the 2d array as data.
feat_df = pd.DataFrame(X,index=range(X.shape[0]),columns=col)
# Save the feature dataframe
os.makedirs(osp.join(DS_FOLDER,'features',model),exist_ok=True)
display(feat_df)

feat_df.loc[~pd.isna(df['North']), :].to_csv(osp.join(DS_FOLDER,'features',model, 'trainval.csv'),index=False)
feat_df.loc[pd.isna(df['North']), :].to_csv(osp.join(DS_FOLDER,'features',model, 'test.csv'),index=False)

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  6.41it/s]


,mat_0,mat_1,mat_2,mat_3,mat_4,mat_5,mat_6,mat_7,mat_8,ef0_0,...,ef1_6,ef1_7,ef1_8,ef1_9,ef1_10,conf_0,conf_1,conf_2,conf_3,conf_4
0,0.997199,0.001646,2.977600,-0.000095,0.998869,1.308397,-5.093368e-06,-0.000002,1.0,-5.067490,...,-1.250218,-1.242380,-1.231876,-1.209067,-0.735909,0.300139,0.572847,0.572869,0.572871,0.572872
1,1.005845,0.000628,0.191164,0.002976,1.005789,-1.129162,6.401204e-05,0.000004,1.0,-1.495178,...,0.917903,0.959494,1.018648,1.114135,2.080652,0.295980,0.563280,0.571988,0.572827,0.572872
2,1.002666,-0.000808,-1.642501,0.001741,1.000746,3.786195,2.257165e-05,-0.000010,1.0,0.120789,...,-3.890309,-3.870249,-3.844637,-3.803320,-3.251992,0.462080,0.572859,0.572868,0.572871,0.572872
3,1.020131,0.010518,-3.981294,-0.010265,1.006860,6.580093,2.453464e-06,0.000079,1.0,1.244383,...,-5.783880,-5.661088,-5.581955,-5.453252,-4.195916,0.391697,0.572859,0.572870,0.572872,0.572872
4,0.996602,-0.003134,8.360514,-0.001560,0.998482,0.758233,-3.319132e-05,0.000026,1.0,-9.995064,...,-0.533055,-0.499078,-0.467451,-0.436842,-0.106895,0.237026,0.572477,0.572865,0.572870,0.572872
5,1.000646,-0.001438,-0.207467,-0.000240,0.999420,1.297872,9.990387e-06,-0.000015,1.0,-0.234032,...,-1.254985,-1.242882,-1.228972,-1.206512,-0.841086,0.492234,0.572856,0.572867,0.572870,0.572872
6,1.005041,0.001884,-5.048995,0.000024,1.005335,-1.999054,5.190567e-06,0.000027,1.0,4.086952,...,1.833006,1.853075,1.875781,1.935270,3.170885,0.446535,0.572225,0.572825,0.572866,0.572872
7,0.987696,-0.023689,1.705560,0.011023,0.977183,6.166472,1.872593e-08,-0.000192,1.0,-2.536316,...,-6.330072,-6.189418,-6.047063,-5.847726,-5.330689,0.324774,0.572826,0.572863,0.572869,0.572872
8,0.947952,0.002231,10.153619,-0.039127,0.989292,9.931049,-2.589543e-04,0.000008,1.0,-10.984010,...,-7.456038,-7.400121,-7.338771,-7.265960,-6.524284,0.103602,0.555731,0.572159,0.572844,0.572872
9,1.000098,-0.003899,-2.332422,-0.000402,1.000340,-6.612254,-2.508822e-05,-0.000070,1.0,1.397377,...,6.321239,6.389812,6.494902,6.707514,8.986298,0.203598,0.565570,0.572506,0.572865,0.572872


TypeError: exceptions must derive from BaseException